In [32]:
import sys, time
sys.argv = ["main", 6, "Input/data_demarcated.pkl", "test.pkl", "Output/"]

In [64]:
import os, itertools, time, pickle, operator
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re, sys
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

f = open(sys.argv[2], "rb")
aml_data, data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, features_dict, ontologies_in_alignment = pickle.load(f)
ontologies_in_alignment = [tuple(pair) for pair in ontologies_in_alignment]
flatten = lambda l: [item for sublist in l for item in sublist]

direct_inputs, direct_targets = [], []

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a,b)

all_fn, all_fp = [], []

threshold_results = {}

def test():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics, direct_inputs, direct_targets, threshold_results
    all_results = OrderedDict()    
    direct_inputs, direct_targets = [], []
    with torch.no_grad():
        all_pred = []
        
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        inputs_pos, nodes_pos, targets_pos = generate_input(test_data_t, 1)
        inputs_neg, nodes_neg, targets_neg = generate_input(test_data_f, 0)

        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        indices_all = np.random.permutation(len(inputs_all))
        inputs_all = np.array(inputs_all)[indices_all]
        targets_all = np.array(targets_all)[indices_all]
        nodes_all = np.array(nodes_all)[indices_all]

        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            inputs = inputs_all[batch_start: batch_end]
            targets = targets_all[batch_start: batch_end]
            nodes = nodes_all[batch_start: batch_end]
            
            inp_elems = torch.LongTensor(inputs).to(device)
            node_elems = torch.LongTensor(nodes).to(device)
            targ_elems = torch.DoubleTensor(targets)

            outputs = model(node_elems, inp_elems)
            outputs = [el.item() for el in outputs]
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[nodes[idx][0]]
                ent2 = emb_indexer_inv[nodes[idx][1]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
        direct_targets = [True if el else False for el in direct_targets]
        
        print ("Len (direct inputs): ", len(direct_inputs))
        for idx, direct_input in enumerate(direct_inputs):
            ent1 = emb_indexer_inv[direct_input[0]]
            ent2 = emb_indexer_inv[direct_input[1]]
            sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
            all_results[(ent1, ent2)] = (sim, direct_targets[idx])
    return (test_onto, all_results)

def optimize_threshold():
    global batch_size, val_data_t, val_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics, direct_inputs, direct_targets, threshold_results
    all_results = OrderedDict()
    direct_inputs, direct_targets = [], []
    with torch.no_grad():
        all_pred = []
        
        np.random.shuffle(val_data_t)
        np.random.shuffle(val_data_f)

        inputs_pos, nodes_pos, targets_pos = generate_input(val_data_t, 1)
        inputs_neg, nodes_neg, targets_neg = generate_input(val_data_f, 0)

        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        indices_all = np.random.permutation(len(inputs_all))
        inputs_all = np.array(inputs_all)[indices_all]
        nodes_all = np.array(nodes_all)[indices_all]
        targets_all = np.array(targets_all)[indices_all]

        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            inputs = inputs_all[batch_start: batch_end]
            nodes = nodes_all[batch_start: batch_end]
            targets = targets_all[batch_start: batch_end]
            
            inp_elems = torch.LongTensor(inputs).to(device)
            node_elems = torch.LongTensor(nodes).to(device)
            targ_elems = torch.DoubleTensor(targets)

            outputs = model(node_elems, inp_elems)
            outputs = [el.item() for el in outputs]
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[nodes[idx][0]]
                ent2 = emb_indexer_inv[nodes[idx][1]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
        direct_targets = [True if el else False for el in direct_targets]
        
        print ("Len (direct inputs): ", len(direct_inputs))
        for idx, direct_input in enumerate(direct_inputs):
            ent1 = emb_indexer_inv[direct_input[0]]
            ent2 = emb_indexer_inv[direct_input[1]]
            sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
            all_results[(ent1, ent2)] = (sim, direct_targets[idx])
        
        low_threshold = np.min([el[0] for el in all_results.values()]) - 0.02
        high_threshold = np.max([el[0] for el in all_results.values()]) + 0.02
        threshold = low_threshold
        step = 0.001
        while threshold < high_threshold:
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)
            fn_list = [(key, all_results[key][0]) for key in val_data_t if key not in set(res) and not is_valid(val_onto, key)]
            fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
            tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
            
            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
            exception = False
            
            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except Exception as e:
                print (e)
                exception = True
                step = 0.001
                threshold += step
                continue
            # print ("Threshold: ", threshold, precision, recall, f1score, f2score, f0_5score)
            if threshold in threshold_results:
                threshold_results[threshold].append([precision, recall, f1score, f2score, f0_5score])
            else:
                threshold_results[threshold] = [[precision, recall, f1score, f2score, f0_5score]]

            if threshold > 0.98 and not exception:
                step = 0.0001
            else:
                step = 0.001
            threshold += step 

def calculate_performance():
    global final_results
    all_metrics, all_fn, all_fp = [], [], []
    for (test_onto, all_results) in final_results:
        res = []
        for i,key in enumerate(all_results):
            if all_results[key][0] > threshold:
                res.append(key)
        fn_list = [(key, all_results[key][0]) for key in gt_mappings if key not in set(res) and not is_valid(test_onto, key)]
        fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
        tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
        tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
        
        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            print (e)
            continue
        print ("Performance for", test_onto, "is :", (precision, recall, f1score, f2score, f0_5score))
        all_fn.extend(fn_list)
        all_fp.extend(fp_list)
        all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    return all_metrics, all_fn, all_fp


def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)

class SiameseNetwork(nn.Module):
    def __init__(self, emb_vals, features_dict):
        super().__init__() 
        
        self.features_arr = np.array(list(features_dict.values()))
        self.n_neighbours = self.features_arr.shape[1]
        self.max_paths = self.features_arr.shape[2]
        self.max_pathlen = self.features_arr.shape[3]
        self.embedding_dim = np.array(emb_vals).shape[1]
        
        self.name_embedding = nn.Embedding(len(emb_vals), self.embedding_dim)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
        self.name_embedding.weight.requires_grad = False

        self.dropout = dropout
        
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        self.output = nn.Linear(2*self.embedding_dim, 300)
        
        self.v = nn.Parameter(torch.DoubleTensor([1/(self.max_pathlen) for i in range(self.max_pathlen)]))
        self.w_rootpath = nn.Parameter(torch.DoubleTensor([0.25]))
        self.w_children = nn.Parameter(torch.DoubleTensor([0.25]))
        self.w_obj_neighbours = nn.Parameter(torch.DoubleTensor([0.25]))
 
    def forward(self, nodes, features):
        '''
        Arguments:
            - nodes: batch_size * 2
            - features: batch_size * 2 * 4 * max_paths * max_pathlen
        '''
        results = []
        nodes = nodes.permute(1,0) # 2 * batch_size
        features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
        for i in range(2):
            node_emb = self.name_embedding(nodes[i]) # batch_size * 512
            feature_emb = self.name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512
            
            feature_emb_reshaped = feature_emb.permute(0,4,1,2,3).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_paths * self.max_pathlen)
            path_weights = torch.bmm(node_emb[:, None, :], feature_emb_reshaped)
            path_weights = path_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_paths, self.max_pathlen)
            path_weights = torch.sum(path_weights, dim=-1)
            best_path_indices = torch.max(path_weights, dim=-1)[1][(..., ) + (None, ) * 3]
            best_path_indices = best_path_indices.expand(-1, -1, -1, self.max_pathlen,  self.embedding_dim)
            best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
            # Another way: 
            # path_weights = masked_softmax(path_weights)
            # best_path = torch.sum(path_weights[:, :, :, None, None] * feature_emb, dim=2)

            best_path_reshaped = best_path.permute(0,3,1,2).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_pathlen)
            node_weights = torch.bmm(node_emb.unsqueeze(1), best_path_reshaped) # batch_size * 4 * max_pathlen
            node_weights = masked_softmax(node_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_pathlen)) # batch_size * 4 * max_pathlen
            attended_path = node_weights.unsqueeze(-1) * best_path # batch_size * 4 * max_pathlen * 512

            distance_weighted_path = torch.sum((self.v[None,None,:,None] * attended_path), dim=2) # batch_size * 4 * 512

            self.w_data_neighbours = (1-self.w_rootpath-self.w_children-self.w_obj_neighbours)
            context_emb = self.w_rootpath * distance_weighted_path[:,0,:] \
                        + self.w_children * distance_weighted_path[:,1,:] \
                        + self.w_obj_neighbours * distance_weighted_path[:,2,:] \
                        + self.w_data_neighbours * distance_weighted_path[:,3,:]

            contextual_node_emb = torch.cat((node_emb, context_emb), dim=1)
            output_node_emb = self.output(contextual_node_emb)
            results.append(output_node_emb)
        sim = self.cosine_sim_layer(results[0], results[1])
        return sim

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data_neighbourless(elem_tuple):
    return np.vectorize(embedify)(elem_tuple)

def embedify(elem):
    return emb_indexer[elem]

def generate_data(elem_tuple):
    return np.vectorize(embedify)([features_dict[elem] for elem in elem_tuple])

def generate_input(elems, target):
    inputs, targets, nodes = [], [], []
    global direct_inputs, direct_targets
    for elem in list(elems):
        try:
            inputs.append(generate_data(elem))
            nodes.append(generate_data_neighbourless(elem))
            targets.append(target)
        except:
            direct_inputs.append(generate_data_neighbourless(elem))
            direct_targets.append(target)
    return np.array(inputs), np.array(nodes), np.array(targets)

print("Max number of nodes in a path: " + str(sys.argv[1]))

def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])

features_dict = {elem: features_dict[elem][:,:,:int(sys.argv[1])] for elem in features_dict}

data_items = aml_data.items()
np.random.shuffle(list(data_items))
aml_data = OrderedDict(data_items)

print ("Number of entities:", len(aml_data))

all_metrics = []
final_results = []
for i in list(range(0, len(ontologies_in_alignment), 3)):
    
    val_onto = ontologies_in_alignment[i:i+2]
    
    train_data = {elem: aml_data[elem] for elem in aml_data if tuple([el.split("#")[0] for el in elem]) not in val_onto}
    val_data = {elem: aml_data[elem] for elem in aml_data if tuple([el.split("#")[0] for el in elem]) in val_onto}

    print ("Training size:", len(train_data), "Validation size:", len(val_data))
    torch.set_default_dtype(torch.float64)

    train_data_t = [key for key in train_data if train_data[key]]
    train_data_f = [key for key in train_data if not train_data[key]]
    train_data_t = np.repeat(train_data_t, ceil(len(train_data_f)/len(train_data_t)), axis=0)
    train_data_t = train_data_t[:len(train_data_f)].tolist()
    #train_data_f = train_data_f[:int(len(train_data_t))]
#     [:int(0.1*(len(train_data) - len(train_data_t)) )]
    np.random.shuffle(train_data_f)
    
    lr = 0.001
    num_epochs = 50
    weight_decay = 0.001
    batch_size = 32
    dropout = 0.3
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model = SiameseNetwork(emb_vals, features_dict).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        inputs_pos, nodes_pos, targets_pos = generate_input(train_data_t, 1)
        inputs_neg, nodes_neg, targets_neg = generate_input(train_data_f, 0)
        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        indices_all = np.random.permutation(len(inputs_all))
        inputs_all = np.array(inputs_all)[indices_all]
        targets_all = np.array(targets_all)[indices_all]
        nodes_all = np.array(nodes_all)[indices_all]

        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size
            
            inputs = inputs_all[batch_start: batch_end]
            targets = targets_all[batch_start: batch_end]
            nodes = nodes_all[batch_start: batch_end]
            
            inp_elems = torch.LongTensor(inputs).to(device)
            node_elems = torch.LongTensor(nodes).to(device)
            targ_elems = torch.DoubleTensor(targets).to(device)

            optimizer.zero_grad()
            outputs = model(node_elems, inp_elems)

            loss = F.mse_loss(outputs, targ_elems)
            loss.backward()
            optimizer.step()

            if batch_idx%5000 == 0:
                print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

    model.eval()
    
    val_data_t = [key for key in val_data if val_data[key]]
    val_data_f = [key for key in val_data if not val_data[key]]
    
    optimize_threshold()

    sys.stdout.flush()

threshold_results_mean = {el: np.mean(threshold_results[el], axis=0) for el in threshold_results}    
threshold = max(threshold_results_mean.keys(), key=(lambda key: threshold_results_mean[key][2]))

train_data_t = [key for key in aml_data if aml_data[key]]
train_data_f = [key for key in aml_data if not aml_data[key]]
train_data_t = np.repeat(train_data_t, ceil(len(train_data_f)/len(train_data_t)), axis=0)
train_data_t = train_data_t[:len(train_data_f)].tolist()
#train_data_f = train_data_f[:int(len(train_data_t))]
#     [:int(0.1*(len(train_data) - len(train_data_t)) )]
np.random.shuffle(train_data_f)

lr = 0.001
num_epochs = 50
weight_decay = 0.001
batch_size = 32
dropout = 0.3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SiameseNetwork(emb_vals, features_dict).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

for epoch in range(num_epochs):
    inputs_pos, nodes_pos, targets_pos = generate_input(train_data_t, 1)
    inputs_neg, nodes_neg, targets_neg = generate_input(train_data_f, 0)
    inputs_all = list(inputs_pos) + list(inputs_neg)
    targets_all = list(targets_pos) + list(targets_neg)
    nodes_all = list(nodes_pos) + list(nodes_neg)
    
    indices_all = np.random.permutation(len(inputs_all))
    inputs_all = np.array(inputs_all)[indices_all]
    targets_all = np.array(targets_all)[indices_all]
    nodes_all = np.array(nodes_all)[indices_all]

    batch_size = min(batch_size, len(inputs_all))
    num_batches = int(ceil(len(inputs_all)/batch_size))

    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx+1) * batch_size
        
        inputs = inputs_all[batch_start: batch_end]
        targets = targets_all[batch_start: batch_end]
        nodes = nodes_all[batch_start: batch_end]
        
        inp_elems = torch.LongTensor(inputs).to(device)
        node_elems = torch.LongTensor(nodes).to(device)
        targ_elems = torch.DoubleTensor(targets).to(device)

        optimizer.zero_grad()
        outputs = model(node_elems, inp_elems)

        loss = F.mse_loss(outputs, targ_elems)
        loss.backward()
        optimizer.step()

        if batch_idx%5000 == 0:
            print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

model.eval()

torch.save(model.state_dict(), sys.argv[4])

model = SiameseNetwork(emb_vals, features_dict)
model.load_state_dict(torch.load(sys.argv[4]), strict=False)

for i in list(range(0, len(ontologies_in_alignment), 3)):
    test_onto = ontologies_in_alignment[i+2:i+3]
    test_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) in test_onto}

    test_data_t = [key for key in test_data if test_data[key]]
    test_data_f = [key for key in test_data if not test_data[key]]

    final_results.append(test())

all_metrics, all_fn, all_fp = calculate_performance()

print ("Final Results: " + str(np.mean(all_metrics, axis=0)))
print ("Threshold: ", threshold)

f1 = open(sys.argv[-1], "wb")
pickle.dump([all_fn, all_fp], f1)

Max number of nodes in a path: 6
Number of entities: 122893
Training size: 109284 Testing size: 3734
Epoch: 0 Idx: 0 Loss: 0.21311044403240967


KeyboardInterrupt: 

In [9]:
# import os, itertools, time, pickle, operator
# import subprocess
# from xml.dom import minidom
# from collections import Counter, OrderedDict
# from operator import itemgetter
# from scipy import spatial
# from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
# from sklearn.feature_extraction.text import TfidfVectorizer
# import re, sys, random
# import numpy as np
# import scipy.sparse as sp
# import torch
# from torch import nn
# from torch import optim
# import torch.nn.functional as F
# from math import ceil, exp
# from collections import Sequence
# from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# sys.argv = ["main", "Models/anatomy_925.pt", "../Archive/test.pkl"]
# t1 = time.time()
# f = open("Input/data_anatomy_oaei_bagofnbrs.pkl", "rb")
# data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment, max_types, max_paths, max_pathlen = pickle.load(f)
# data = pickle.load(open("../Archive/data.pkl", "rb"))

# max_paths = 20
# max_pathlen = 4

# direct_inputs, direct_targets = [], []

# def cos_sim(a,b):
#     return 1 - spatial.distance.cosine(a,b)

# all_fn, all_fp = [], []

# threshold_results = {}

# def test():
#     global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics, direct_inputs, direct_targets, threshold_results
#     all_results = OrderedDict()    
#     direct_inputs, direct_targets = [], []
#     with torch.no_grad():
#         all_pred = []
        
#         np.random.shuffle(test_data_t)
#         np.random.shuffle(test_data_f)
#         print ("Here")
#         sys.stdout.flush()
        
#         print ("Done with this")
#         sys.stdout.flush()
#         inputs_all = list(inputs_pos) + list(inputs_neg)
#         targets_all = list(targets_pos) + list(targets_neg)
#         nodes_all = list(nodes_pos) + list(nodes_neg)
        
#         all_inp = list(zip(inputs_all, targets_all, nodes_all))
#         all_inp_shuffled = random.sample(all_inp, len(all_inp))
#         inputs_all, targets_all, nodes_all = list(zip(*all_inp_shuffled))
        
#         batch_size = min(batch_size, len(inputs_all))
#         num_batches = int(ceil(len(inputs_all)/batch_size))
#         print ("Total batches:", num_batches)
#         for batch_idx in range(num_batches):
#             batch_start = batch_idx * batch_size
#             batch_end = (batch_idx+1) * batch_size

#             inputs = np.array(to_feature(inputs_all[batch_start: batch_end]))
#             targets = np.array(targets_all[batch_start: batch_end])
#             nodes = np.array(nodes_all[batch_start: batch_end])
            
#             inp_elems = torch.LongTensor(inputs).to(device)
#             node_elems = torch.LongTensor(nodes).to(device)
#             targ_elems = torch.DoubleTensor(targets)

#             t = time.time()
#             outputs = model(node_elems, inp_elems)
# #             if batch_idx % 1000 == 0:
#             print (time.time()-t, "Approx time: ", num_batches * (time.time()-t))
#             sys.stdout.flush()
#             outputs = [el.item() for el in outputs]
#             targets = [True if el.item() else False for el in targets]

#             for idx, pred_elem in enumerate(outputs):
#                 ent1 = emb_indexer_inv[nodes[idx][0]]
#                 ent2 = emb_indexer_inv[nodes[idx][1]]
#                 if (ent1, ent2) in all_results:
#                     print ("Error: ", ent1, ent2, "already present")
#                 all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
#         direct_targets = [True if el else False for el in direct_targets]
        
#         print ("Len (direct inputs): ", len(direct_inputs))
#         sys.stdout.flush()
#         for idx, direct_input in enumerate(direct_inputs):
#             ent1 = emb_indexer_inv[direct_input[0]]
#             ent2 = emb_indexer_inv[direct_input[1]]
#             sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
#             all_results[(ent1, ent2)] = (sim, direct_targets[idx])
#         print ("Done!")
#         sys.stdout.flush()
#     return (test_data_t, all_results)

# def calculate_performance():
#     global final_results
#     all_metrics, all_fn, all_fp = [], [], []
#     print ("Calculating performance....")
#     for (test_data_t, all_results) in final_results:
#         res = []
#         for i,key in enumerate(all_results):
#             if all_results[key][0] > threshold:
#                 res.append(key)
#         s = set(res)
#         fn_list = [(key, all_results[key][0]) for key in test_data_t if key not in s]
#         fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
#         tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
#         tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
        
#         try:
#             precision = tp/(tp+fp)
#             recall = tp/(tp+fn)
#             f1score = 2 * precision * recall / (precision + recall)
#             f2score = 5 * precision * recall / (4 * precision + recall)
#             f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
#         except Exception as e:
#             # print (e)
#             continue
#         all_fn.extend(fn_list)
#         all_fp.extend(fp_list)
#         all_metrics.append((precision, recall, f1score, f2score, f0_5score))
#     print ("Done..")
#     return all_metrics, all_fn, all_fp


# def masked_softmax(inp):
#     inp = inp.double()
#     mask = ((inp != 0).double() - 1) * 9999  # for -inf
#     return (inp + mask).softmax(dim=-1)

# class SiameseNetwork(nn.Module):
#     def __init__(self, emb_vals, threshold=0.9):
#         super().__init__() 
        
#         self.n_neighbours = max_types
#         self.max_paths = max_paths
#         self.max_pathlen = max_pathlen
#         self.embedding_dim = np.array(emb_vals).shape[1]
        
#         self.threshold = nn.Parameter(torch.DoubleTensor([threshold]))
#         self.threshold.requires_grad = False

#         self.name_embedding = nn.Embedding(len(emb_vals), self.embedding_dim)
#         self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
#         self.name_embedding.weight.requires_grad = False

#         self.dropout = dropout
        
#         self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
#         self.output = nn.Linear(2*self.embedding_dim, 300)
        
#         self.v = nn.Parameter(torch.DoubleTensor([1/(self.max_pathlen) for i in range(self.max_pathlen)]))
#         self.w_rootpath = nn.Parameter(torch.DoubleTensor([0.33]))
#         self.w_children = nn.Parameter(torch.DoubleTensor([0.33]))
 
#     def forward(self, nodes, features):
#         '''
#         Arguments:
#             - nodes: batch_size * 2
#             - features: batch_size * 2 * 4 * max_paths * max_pathlen
#         '''
#         results = []
#         nodes = nodes.permute(1,0) # 2 * batch_size
#         features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
#         for i in range(2):
#             node_emb = self.name_embedding(nodes[i]) # batch_size * 512
#             feature_emb = self.name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512
            
#             feature_emb_reshaped = feature_emb.permute(0,4,1,2,3).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_paths * self.max_pathlen)
#             path_weights = torch.bmm(node_emb[:, None, :], feature_emb_reshaped)
#             path_weights = path_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_paths, self.max_pathlen)
#             path_weights = torch.sum(path_weights, dim=-1)
#             best_path_indices = torch.max(path_weights, dim=-1)[1][(..., ) + (None, ) * 3]
#             best_path_indices = best_path_indices.expand(-1, -1, -1, self.max_pathlen,  self.embedding_dim)
#             best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
#             # Another way: 
#             # path_weights = masked_softmax(path_weights)
#             # best_path = torch.sum(path_weights[:, :, :, None, None] * feature_emb, dim=2)

#             best_path_reshaped = best_path.permute(0,3,1,2).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_pathlen)
#             node_weights = torch.bmm(node_emb.unsqueeze(1), best_path_reshaped) # batch_size * 4 * max_pathlen
#             node_weights = masked_softmax(node_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_pathlen)) # batch_size * 4 * max_pathlen
#             attended_path = node_weights.unsqueeze(-1) * best_path # batch_size * 4 * max_pathlen * 512

#             distance_weighted_path = torch.sum((self.v[None,None,:,None] * attended_path), dim=2) # batch_size * 4 * 512

#             self.w_obj_neighbours = (1-self.w_rootpath-self.w_children)
#             context_emb = self.w_rootpath * distance_weighted_path[:,0,:] \
#                         + self.w_children * distance_weighted_path[:,1,:] \
#                         + self.w_obj_neighbours * distance_weighted_path[:,2,:]

#             contextual_node_emb = torch.cat((node_emb, context_emb), dim=1)
#             output_node_emb = self.output(contextual_node_emb)
#             results.append(output_node_emb)
#         sim = self.cosine_sim_layer(results[0], results[1])
#         return sim

# def is_valid(test_onto, key):
#     return tuple([el.split("#")[0] for el in key]) not in test_onto

# def generate_data_neighbourless(elem_tuple):
#     return [emb_indexer[elem] for elem in elem_tuple]

# def embedify(seq, emb_indexer):
#     for item in seq:
#         if isinstance(item, list):
#             yield list(embedify(item, emb_indexer))
#         else:
#             yield emb_indexer[item]

# def generate_data(elem_tuple):
#     return list(embedify([neighbours_dicts[elem] for elem in elem_tuple], emb_indexer))

# def to_feature(inputs):
    
# #     print ("Step I: Sliced")
#     sys.stdout.flush()
#     t = time.time()
#     inputs_lenpadded = [[[[path[:max_pathlen]
#                                     for path in nbr_type[:max_paths]]
#                                 for nbr_type in ent[:max_types]]
#                             for ent in elem]
#                         for elem in inputs]
# #     print ("Step II: Zero padding {}".format(time.time() - t))
#     sys.stdout.flush()
#     t = time.time()
#     inputs_lenpadded = [[[[path + [0 for i in range(max_pathlen -len(path))]
#                                     for path in nbr_type]
#                                 for nbr_type in ent]
#                             for ent in elem]
#                         for elem in inputs_lenpadded]
# #     print ("Step III: Path padding {}".format(time.time() - t))
#     sys.stdout.flush()
#     t = time.time()
#     inputs_pathpadded = [[[nbr_type + [[0 for j in range(max_pathlen)]
#                              for i in range(max_paths - len(nbr_type))]
#                             for nbr_type in ent] for ent in elem]
#                         for elem in inputs_lenpadded]
# #     print ("Step IV: Done {}".format(time.time() - t))
#     sys.stdout.flush()
#     t = time.time()
#     return inputs_pathpadded

# def generate_input(elems, target):
#     inputs, targets, nodes = [], [], []
#     global direct_inputs, direct_targets
#     for elem in list(elems):
#         try:
#             inputs.append(generate_data(elem))
#             nodes.append(generate_data_neighbourless(elem))
#             targets.append(target)
#         except:
#             direct_inputs.append(generate_data_neighbourless(elem))
#             direct_targets.append(target)
#     return inputs, nodes, targets

# torch.set_default_dtype(torch.float64)

# lr = 0.001
# num_epochs = 50
# weight_decay = 0.001
# batch_size = 512
# dropout = 0.3
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model = SiameseNetwork(emb_vals).to(device)
# model.load_state_dict(torch.load(sys.argv[1], map_location=device), strict=False)

# all_metrics = []
# final_results = []

# threshold = model.threshold.data.cpu().numpy()[0]

# test_data_t = [key for key in data if data[key]]
# test_data_f = [key for key in data if not data[key]]

# inputs_pos, nodes_pos, targets_pos = generate_input(test_data_t, 1)
# inputs_neg, nodes_neg, targets_neg = generate_input(test_data_f, 0)
batch_size = 10000    
final_results.append(test())

print ("Total time: ", time.time()-t1)

all_metrics, all_fn, all_fp = calculate_performance()

print ("Final Results: " + str(np.mean(all_metrics, axis=0)))
print ("Threshold: ", threshold)

f1 = open(sys.argv[2], "wb")
pickle.dump([all_fn, all_fp], f1)



Here
Done with this
Total batches: 903
6.534803867340088 Approx time:  5900.934566259384
6.453084945678711 Approx time:  5827.138074159622
5.803795576095581 Approx time:  5240.829988718033
6.291790246963501 Approx time:  5681.489822387695
5.717963933944702 Approx time:  5163.33025932312
5.798051118850708 Approx time:  5235.642528533936
6.658377170562744 Approx time:  6012.516737937927
5.882312774658203 Approx time:  5311.734033107758
5.927946090698242 Approx time:  5352.937688112259
5.938767671585083 Approx time:  5362.712589740753
6.588241338729858 Approx time:  5949.183220624924
6.132535696029663 Approx time:  5537.681886434555
6.415138244628906 Approx time:  5792.873279571533
5.066014528274536 Approx time:  4574.624682426453
5.09234094619751 Approx time:  4598.3862426280975
5.493236303329468 Approx time:  4960.3949654102325
5.253589868545532 Approx time:  4743.994450092316
5.126105308532715 Approx time:  4628.878045320511
5.0384016036987305 Approx time:  4549.684829235077
5.22915959

5.206872224807739 Approx time:  4701.809924840927
5.2164812088012695 Approx time:  4710.485976219177
5.499050140380859 Approx time:  4965.644860267639
5.268618822097778 Approx time:  4757.568393945694
6.3724236488342285 Approx time:  5754.309965372086
6.124371767044067 Approx time:  5530.318254947662
6.155117034912109 Approx time:  5558.073266029358
5.883415699005127 Approx time:  5312.726959705353
6.083115816116333 Approx time:  5493.058318376541
5.605215311050415 Approx time:  5061.517176389694
6.300387859344482 Approx time:  5689.254542827606
6.124422073364258 Approx time:  5530.360236883163
5.617161750793457 Approx time:  5072.30136680603
5.749609470367432 Approx time:  5191.920172691345
6.372994661331177 Approx time:  5754.81719326973
5.824108839035034 Approx time:  5259.175448656082
5.833110332489014 Approx time:  5267.303797245026
5.917140960693359 Approx time:  5343.183239221573
6.2807910442352295 Approx time:  5671.557327032089
6.368102073669434 Approx time:  5750.399186611175

6.086988687515259 Approx time:  5496.562195301056
6.4523468017578125 Approx time:  5826.48143362999
7.0804219245910645 Approx time:  6393.623366117477
6.53463339805603 Approx time:  5900.778048992157
6.498794794082642 Approx time:  5868.413851976395
6.827328205108643 Approx time:  6165.0887796878815
6.739951848983765 Approx time:  6086.178672552109
7.173361301422119 Approx time:  6477.547408103943
6.647632598876953 Approx time:  6002.8156814575195
6.520042657852173 Approx time:  5887.601749420166
6.628647565841675 Approx time:  5985.671981334686
6.916324853897095 Approx time:  6245.450170040131
6.862654209136963 Approx time:  6196.979980230331
6.949903249740601 Approx time:  6275.7652180194855
7.010709762573242 Approx time:  6330.673283815384
6.6442601680755615 Approx time:  5999.7686541080475
6.5720765590667725 Approx time:  5934.592668056488
6.6132001876831055 Approx time:  5971.722783565521
7.017572641372681 Approx time:  6336.874984502792
6.5098676681518555 Approx time:  5878.42213

6.047715902328491 Approx time:  5461.090043306351
6.114595174789429 Approx time:  5521.482241630554
6.406411409378052 Approx time:  5785.0024201869965
6.4582014083862305 Approx time:  5831.762545824051
6.771883726119995 Approx time:  6115.013803482056
6.468982934951782 Approx time:  5841.494604349136
6.413093328475952 Approx time:  5791.0271508693695
6.514364719390869 Approx time:  5882.474786281586
6.321107864379883 Approx time:  5707.963415622711
6.86346697807312 Approx time:  6197.71541762352
6.8458571434021 Approx time:  6181.811153411865
6.311415910720825 Approx time:  5699.213088512421
6.472330331802368 Approx time:  5844.516873121262
6.6495630741119385 Approx time:  6004.558039426804
6.514255523681641 Approx time:  5882.377259016037
6.613598108291626 Approx time:  5972.0812447071075
6.6911303997039795 Approx time:  6042.101084947586
6.707201719284058 Approx time:  6056.609611272812
6.58222222328186 Approx time:  5943.749681711197
6.455456018447876 Approx time:  5829.278506994247

6.156992435455322 Approx time:  5559.772135019302
6.385725498199463 Approx time:  5766.317444801331
6.71235203742981 Approx time:  6061.256688594818
6.759509086608887 Approx time:  6103.840580463409
6.060707330703735 Approx time:  5472.825608968735
6.487161636352539 Approx time:  5857.91277050972
6.35250449180603 Approx time:  5736.314570188522
6.793983697891235 Approx time:  6134.972661495209
6.613958358764648 Approx time:  5972.40633559227
6.766222715377808 Approx time:  6109.909230709076
6.703090190887451 Approx time:  6052.893025875092
6.439784049987793 Approx time:  5815.127365350723
6.335410118103027 Approx time:  5720.8789966106415
6.663946628570557 Approx time:  6017.546604394913
6.43507981300354 Approx time:  5810.878578186035
5.8254382610321045 Approx time:  5260.383021354675
6.595219373703003 Approx time:  5955.484816789627
6.572096824645996 Approx time:  5934.605800867081
6.617831707000732 Approx time:  5975.905906677246
6.744765758514404 Approx time:  6090.5252022743225
6.

6.567864894866943 Approx time:  5930.784583568573
6.358100414276123 Approx time:  5741.3694105148315
6.209634780883789 Approx time:  5607.306665897369
6.7468178272247314 Approx time:  6092.379081487656
6.753769874572754 Approx time:  6098.656349658966
6.706660270690918 Approx time:  6056.116807937622
6.412899494171143 Approx time:  5790.854271411896
6.456035375595093 Approx time:  5829.802527666092
6.469054222106934 Approx time:  5841.562421321869
6.3447184562683105 Approx time:  5729.283349514008
6.69661283493042 Approx time:  6047.0435428619385
6.7549147605896 Approx time:  6099.690181732178
6.886715888977051 Approx time:  6218.707246541977
7.06231689453125 Approx time:  6377.276461601257
6.308162212371826 Approx time:  5696.273061275482
6.459813594818115 Approx time:  5833.214259624481
6.480188608169556 Approx time:  5851.615049600601
6.4601829051971436 Approx time:  5833.547962188721
6.768903970718384 Approx time:  6112.327605485916
6.9200029373168945 Approx time:  6248.76459002494

In [11]:
all_fp

[(('mouse#nail plate', 'human#Nail'), 0.8204742092231412),
 (('mouse#adrenal gland cortex zone', 'human#Adrenal_Gland'),
  0.820395104255702),
 (('mouse#metacarpal bone digit 2', 'human#Metacarpal_Bone_Digit_4'),
  0.9679550398998976),
 (('mouse#tibialis muscle', 'human#Muscle'), 0.7962570975346701),
 (('mouse#uterus', 'human#Ovary'), 0.7787046103613111),
 (('mouse#metatarsal bone digit 3', 'human#Metacarpal_Bone_Digit_2'),
  0.7983945198082326),
 (('mouse#palpebral fissure', 'human#Postero-lateral_Fissure'),
  0.8473946290129749),
 (('mouse#olfactory lobe', 'human#Olfactory_Tract'), 0.8053237305761524),
 (('mouse#supraoptic nucleus', 'human#Nucleus_Ambiguus'), 0.875290193075474),
 (('mouse#retina inner nuclear layer', 'human#Outer_Nuclear_Layer_of_Retina'),
  0.9736868377623888),
 (('mouse#external jugular vein', 'human#Internal_Jugular_Vein'),
  0.9372127818408752),
 (('mouse#anus', 'human#Sphincter'), 0.8397837054720046),
 (('mouse#urinary bladder wall', 'human#Lateral_Wall_of_the_B

In [13]:
import os, itertools, time, pickle, operator
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re, sys, random
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from collections import Sequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

sys.argv = ["main", "Output/test_anatomy_aml_bagofnbrs10_14.pkl", "../Archive/test.pkl"]
t1 = time.time()
f = open("Input/data_anatomy_oaei_bagofnbrs.pkl", "rb")
data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment, max_types, max_paths, max_pathlen = pickle.load(f)
data = pickle.load(open("../Archive/data.pkl", "rb"))

max_paths = 10
max_pathlen = 14

direct_inputs, direct_targets = [], []

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a,b)

all_fn, all_fp = [], []

threshold_results = {}

def test():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics, direct_inputs, direct_targets, threshold_results
    all_results = OrderedDict()    
    direct_inputs, direct_targets = [], []
    with torch.no_grad():
        all_pred = []
        
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)
        print ("Here")
        sys.stdout.flush()
        
        print ("Done with this")
        sys.stdout.flush()
        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        all_inp = list(zip(inputs_all, targets_all, nodes_all))
        all_inp_shuffled = random.sample(all_inp, len(all_inp))
        inputs_all, targets_all, nodes_all = list(zip(*all_inp_shuffled))
        
        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))
        print ("Total batches:", num_batches)
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            inputs = np.array(to_feature(inputs_all[batch_start: batch_end]))
            targets = np.array(targets_all[batch_start: batch_end])
            nodes = np.array(nodes_all[batch_start: batch_end])
            
            inp_elems = torch.LongTensor(inputs).to(device)
            node_elems = torch.LongTensor(nodes).to(device)
            targ_elems = torch.DoubleTensor(targets)

            t = time.time()
            outputs = model(node_elems, inp_elems)
#             if batch_idx % 1000 == 0:
            print (time.time()-t, "Approx time: ", num_batches * (time.time()-t))
            sys.stdout.flush()
            outputs = [el.item() for el in outputs]
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[nodes[idx][0]]
                ent2 = emb_indexer_inv[nodes[idx][1]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
        direct_targets = [True if el else False for el in direct_targets]
        
        print ("Len (direct inputs): ", len(direct_inputs))
        sys.stdout.flush()
        for idx, direct_input in enumerate(direct_inputs):
            ent1 = emb_indexer_inv[direct_input[0]]
            ent2 = emb_indexer_inv[direct_input[1]]
            sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
            all_results[(ent1, ent2)] = (sim, direct_targets[idx])
        print ("Done!")
        sys.stdout.flush()
    return (test_data_t, all_results)

def calculate_performance():
    global final_results
    all_metrics, all_fn, all_fp = [], [], []
    print ("Calculating performance....")
    for (test_data_t, all_results) in final_results:
        res = []
        for i,key in enumerate(all_results):
            if all_results[key][0] > threshold:
                res.append(key)
        s = set(res)
        fn_list = [(key, all_results[key][0]) for key in test_data_t if key not in s]
        fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
        tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
        tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
        
        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            # print (e)
            continue
        all_fn.extend(fn_list)
        all_fp.extend(fp_list)
        all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    print ("Done..")
    return all_metrics, all_fn, all_fp


def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)

class SiameseNetwork(nn.Module):
    def __init__(self, emb_vals, threshold=0.9):
        super().__init__() 
        
        self.n_neighbours = max_types
        self.max_paths = max_paths
        self.max_pathlen = max_pathlen
        self.embedding_dim = np.array(emb_vals).shape[1]
        
        self.threshold = nn.Parameter(torch.DoubleTensor([threshold]))
        self.threshold.requires_grad = False

        self.name_embedding = nn.Embedding(len(emb_vals), self.embedding_dim)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
        self.name_embedding.weight.requires_grad = False

        self.dropout = dropout
        
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        self.output = nn.Linear(2*self.embedding_dim, 300)
        
        self.v = nn.Parameter(torch.DoubleTensor([1/(self.max_pathlen) for i in range(self.max_pathlen)]))
        self.w_rootpath = nn.Parameter(torch.DoubleTensor([0.33]))
        self.w_children = nn.Parameter(torch.DoubleTensor([0.33]))
 
    def forward(self, nodes, features):
        '''
        Arguments:
            - nodes: batch_size * 2
            - features: batch_size * 2 * 4 * max_paths * max_pathlen
        '''
        results = []
        nodes = nodes.permute(1,0) # 2 * batch_size
        features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
        for i in range(2):
            node_emb = self.name_embedding(nodes[i]) # batch_size * 512
            feature_emb = self.name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512
            
            feature_emb_reshaped = feature_emb.permute(0,4,1,2,3).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_paths * self.max_pathlen)
            path_weights = torch.bmm(node_emb[:, None, :], feature_emb_reshaped)
            path_weights = path_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_paths, self.max_pathlen)
            path_weights = torch.sum(path_weights, dim=-1)
            best_path_indices = torch.max(path_weights, dim=-1)[1][(..., ) + (None, ) * 3]
            best_path_indices = best_path_indices.expand(-1, -1, -1, self.max_pathlen,  self.embedding_dim)
            best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
            # Another way: 
            # path_weights = masked_softmax(path_weights)
            # best_path = torch.sum(path_weights[:, :, :, None, None] * feature_emb, dim=2)

            best_path_reshaped = best_path.permute(0,3,1,2).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_pathlen)
            node_weights = torch.bmm(node_emb.unsqueeze(1), best_path_reshaped) # batch_size * 4 * max_pathlen
            node_weights = masked_softmax(node_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_pathlen)) # batch_size * 4 * max_pathlen
            attended_path = node_weights.unsqueeze(-1) * best_path # batch_size * 4 * max_pathlen * 512

            distance_weighted_path = torch.sum((self.v[None,None,:,None] * attended_path), dim=2) # batch_size * 4 * 512

            self.w_obj_neighbours = (1-self.w_rootpath-self.w_children)
            context_emb = self.w_rootpath * distance_weighted_path[:,0,:] \
                        + self.w_children * distance_weighted_path[:,1,:] \
                        + self.w_obj_neighbours * distance_weighted_path[:,2,:]

            contextual_node_emb = torch.cat((node_emb, context_emb), dim=1)
            output_node_emb = self.output(contextual_node_emb)
            results.append(output_node_emb)
        sim = self.cosine_sim_layer(results[0], results[1])
        return sim

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data_neighbourless(elem_tuple):
    return [emb_indexer[elem] for elem in elem_tuple]

def embedify(seq, emb_indexer):
    for item in seq:
        if isinstance(item, list):
            yield list(embedify(item, emb_indexer))
        else:
            yield emb_indexer[item]

def generate_data(elem_tuple):
    return list(embedify([neighbours_dicts[elem] for elem in elem_tuple], emb_indexer))

def to_feature(inputs):
    
#     print ("Step I: Sliced")
    sys.stdout.flush()
    t = time.time()
    inputs_lenpadded = [[[[path[:max_pathlen]
                                    for path in nbr_type[:max_paths]]
                                for nbr_type in ent[:max_types]]
                            for ent in elem]
                        for elem in inputs]
#     print ("Step II: Zero padding {}".format(time.time() - t))
    sys.stdout.flush()
    t = time.time()
    inputs_lenpadded = [[[[path + [0 for i in range(max_pathlen -len(path))]
                                    for path in nbr_type]
                                for nbr_type in ent]
                            for ent in elem]
                        for elem in inputs_lenpadded]
#     print ("Step III: Path padding {}".format(time.time() - t))
    sys.stdout.flush()
    t = time.time()
    inputs_pathpadded = [[[nbr_type + [[0 for j in range(max_pathlen)]
                             for i in range(max_paths - len(nbr_type))]
                            for nbr_type in ent] for ent in elem]
                        for elem in inputs_lenpadded]
#     print ("Step IV: Done {}".format(time.time() - t))
    sys.stdout.flush()
    t = time.time()
    return inputs_pathpadded

def generate_input(elems, target):
    inputs, targets, nodes = [], [], []
    global direct_inputs, direct_targets
    for elem in list(elems):
        try:
            inputs.append(generate_data(elem))
            nodes.append(generate_data_neighbourless(elem))
            targets.append(target)
        except:
            direct_inputs.append(generate_data_neighbourless(elem))
            direct_targets.append(target)
    return inputs, nodes, targets

torch.set_default_dtype(torch.float64)

lr = 0.001
num_epochs = 50
weight_decay = 0.001
batch_size = 512
dropout = 0.3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SiameseNetwork(emb_vals).to(device)
model.load_state_dict(torch.load(sys.argv[1], map_location=device), strict=False)

all_metrics = []
final_results = []

threshold = model.threshold.data.cpu().numpy()[0]

test_data_t = [key for key in data if data[key]]
test_data_f = [key for key in data if not data[key]]

inputs_pos, nodes_pos, targets_pos = generate_input(test_data_t, 1)
inputs_neg, nodes_neg, targets_neg = generate_input(test_data_f, 0)
batch_size = 10000    
final_results.append(test())

print ("Total time: ", time.time()-t1)

all_metrics, all_fn, all_fp = calculate_performance()

print ("Final Results: " + str(np.mean(all_metrics, axis=0)))
print ("Threshold: ", threshold)

f1 = open(sys.argv[2], "wb")
pickle.dump([all_fn, all_fp], f1)




RuntimeError: Invalid magic number; corrupt file?

In [59]:
def extractUSEEmbeddings(words):
    # Extracts USE embeddings
    try:
        embed = hub.KerasLayer("/home/vlead/USE")
    except Exception as e:
        print (e)
        print ("Downloading USE embeddings...")
        r = requests.get("https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed")
        open("USE.tar.gz", "wb").write(r.content)

        tar = tarfile.open("USE.tar.gz", "r:gz")
        tar.extractall(path="/home/vlead/USE")
        tar.close()

        os.remove("USE.tar.gz")
        embed = hub.KerasLayer("/home/vlead/USE")
        pass
    word_embeddings = embed(words)
    print (type(word_embeddings))
    return tf.constant(word_embeddings)


In [86]:
torch.load("../Archive/Models/anatomy_aml_bagofnbrs3_2.pt", map_location=device)

OrderedDict([('threshold', tensor([0.8201])),
             ('v', tensor([-0.0130,  0.0463])),
             ('w_rootpath', tensor([-0.0015])),
             ('w_children', tensor([-0.0032])),
             ('name_embedding.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [-0.0171,  0.0269, -0.0159,  ...,  0.0470, -0.0461, -0.0034],
                      [ 0.0596, -0.0706, -0.0414,  ...,  0.0193,  0.0300, -0.0496],
                      ...,
                      [-0.0535, -0.0396, -0.0349,  ...,  0.0008, -0.0019,  0.0260],
                      [ 0.0605, -0.0193, -0.1038,  ..., -0.0126, -0.0320, -0.0342],
                      [ 0.0421, -0.0359,  0.0155,  ...,  0.0176,  0.0640, -0.0010]])),
             ('output.weight',
              tensor([[-6.5158e-03,  9.4744e-03, -7.4258e-03,  ...,  1.4233e-04,
                       -1.3989e-03, -9.2190e-05],
                      [ 7.8852e-03,  3.3722e-03, -8.8502e-03,  ...,  1.5746

In [12]:
len(inputs_all)

169534